# Solver 3D
3D rendering of solver results

In [1]:
import matplotlib.pyplot as plt
import numpy as np

from amdt import Simulator, Solver
from matplotlib.animation import FuncAnimation, PillowWriter
from matplotlib.colors import Normalize
from matplotlib.cm import get_cmap, ScalarMappable
from tqdm import tqdm
from pprint import pprint

In [2]:
s = Simulator()
gcode_commands = s.load_gcode_commands("3DBenchy.gcode")

100%|██████████| 86539/86539 [00:04<00:00, 18859.92it/s]


In [3]:
LAYER_INDEX = 10
START_INDEX = 0
END_INDEX = -1

In [6]:
gcode_layer_commands = s.get_gcode_commands_by_layer_change_index(LAYER_INDEX)
gcode_segments = s.gcode_commands_to_segments(gcode_layer_commands)
first_location = [gcode_segments[0]["X"][0]/1000, gcode_segments[0]["Y"][0]/1000]

100%|██████████| 298/298 [00:00<00:00, 325072.20it/s]


In [7]:
mesh = {
    "b_c": "temp",
    "x_location": first_location[0],
    "y_location": first_location[1],
    "x_min": 0.06,
    "x_max": 0.12,
    "y_min": 0.08,
    "y_max": 0.12,
    "z_min": -800e-6,
    "z_max": 0,
    "x_start_pad": 200e-6, 
    "y_start_pad": 200e-6,
    "x_end_pad": 200e-6, 
    "y_end_pad": 200e-6,
}
solver_et = Solver(mesh=mesh, verbose=True)
solver_r = Solver(mesh=mesh, verbose=True)

velocity = 800 # mm/s

thetas_r = []
thetas_et = []

for index in tqdm(range(len(gcode_segments))):

    power = 200

    line = gcode_segments[index]
    dt = line["distance_xy"] / velocity

    if not line["travel"]:
        power = 0

    solver_r.forward({"dt": dt, "phi": line["angle_xy"], "power": power}, model="rosenthal")
    thetas_r.append(solver_r.theta)

    solver_et.forward({"dt": dt, "phi": line["angle_xy"], "power": power})
    thetas_et.append(solver_et.theta)


Build Configuration
{'beam_diameter': 5e-05, 'power': 200.0, 't_0': 300.0, 'velocity': 0.8}

Material Configuration
{'absorptivity': 1.0,
 'c_p': 455.0,
 'k': 8.9,
 'rho': 7910.0,
 't_liquidus': 1710.26,
 't_melt': 1673.0,
 't_solidus': 1683.68}

Mesh Configuration
{'b_c': 'temp',
 'x_end_pad': 0.0002,
 'x_location': 0.093946,
 'x_max': 0.12,
 'x_min': 0.06,
 'x_start_pad': 0.0002,
 'x_step': 0.0001,
 'y_end_pad': 0.0002,
 'y_location': 0.091817,
 'y_max': 0.12,
 'y_min': 0.08,
 'y_start_pad': 0.0002,
 'y_step': 0.0001,
 'z_end_pad': 0.0001,
 'z_location': 0.0,
 'z_max': 0,
 'z_min': -0.0008,
 'z_start_pad': 0.0,
 'z_step': 0.0001}

Build Configuration
{'beam_diameter': 5e-05, 'power': 200.0, 't_0': 300.0, 'velocity': 0.8}

Material Configuration
{'absorptivity': 1.0,
 'c_p': 455.0,
 'k': 8.9,
 'rho': 7910.0,
 't_liquidus': 1710.26,
 't_melt': 1673.0,
 't_solidus': 1683.68}

Mesh Configuration
{'b_c': 'temp',
 'x_end_pad': 0.0002,
 'x_location': 0.093946,
 'x_max': 0.12,
 'x_min': 0.0

100%|██████████| 298/298 [05:19<00:00,  1.07s/it]


In [2]:
np.savez("thetas_r.npz", data = thetas_r)
np.savez("thetas_et.npz", data = thetas_et)

NameError: name 'thetas_r' is not defined

In [4]:
loaded_thetas_r = np.load("thetas_r.npz")
loaded_thetas_et = np.load("thetas_et.npz")

In [5]:
thetas_r = loaded_thetas_r["data"]
print(len(thetas_r))
print(thetas_r[0].shape)

thetas_et = loaded_thetas_et["data"]
print(len(thetas_et))
print(thetas_et[0].shape)

298
(604, 405, 9)
298
(604, 405, 9)


In [ ]:
key = "temperature"
# title = f"{power} W, {velocity} m/s"
title = "rosenthal"

# values = solver.theta
# print(values.shape)
# mesh_x, mesh_y, mesh_z = np.meshgrid(solver.xs, solver.ys, solver.zs, indexing="ij")
# mesh = np.stack((mesh_x, mesh_y, mesh_z), axis = -1)
# print(mesh.shape)

clim = [300, 3000]
threshold = clim[0]
voxels = thetas_r[0]
# voxels = solver.theta > threshold  # Apply threshold to create a binary voxel structure

cmap = plt.get_cmap("plasma")

# Normalize your data for colormap
norm = Normalize(vmin=clim[0], vmax=clim[1])

colors = cmap(norm(voxels))

# Plot using Matplotlib's voxels
fig = plt.figure(figsize=(10, 10))
ax = fig.add_subplot(111, projection='3d')
ax.voxels(
    voxels,
    facecolors=colors,
    # edgecolors=np.clip(2 * normalized_colors - 0.5, 0, 1),
    linewidth=0.5
)

x_dim, y_dim, z_dim = voxels.shape  # Dimensions of the voxel grid

ax.set_xlim([0, x_dim])
ax.set_ylim([0, y_dim])
ax.set_zlim([0, z_dim])

# Set tick markers every 10 units
ax.set_xticks(np.arange(0, x_dim + 1, 10))
ax.set_yticks(np.arange(0, y_dim + 1, 10))
ax.set_zticks(np.arange(0, z_dim + 1, 10))

ax.set(xlabel='X', ylabel='Y', zlabel='Z')
ax.set_title(title)

# Calculate the maximum extent for equal aspect ratio
max_extent = max(x_dim, y_dim, z_dim)

# Center and scale each axis to have equal aspect ratio
ax.set_box_aspect((x_dim / max_extent, y_dim / max_extent, z_dim / max_extent))
# ax.set_aspect('auto')

# Add color bar
mappable = ScalarMappable(norm=norm, cmap=cmap)
mappable.set_array([])  # This line is necessary to avoid errors

# Add the color bar to the figure
cbar = fig.colorbar(mappable, ax=ax, shrink=0.5, aspect=10)
cbar.set_label(key)


KeyboardInterrupt: 